# 네이버 검색 API 를 통한 뉴스 가져오기

[네이버 뉴스 api](https://developers.naver.com/docs/serviceapi/search/news/news.md#%EB%89%B4%EC%8A%A4)

In [1]:
import os
import sys
import urllib.request
import json
import pandas as pd
import re

client_id = input('client_id : ')
client_secret = input('client_secret : ')

query = urllib.parse.quote(input('검색 키워드 : '))

idx = 0
display = 100
start = 1 
end = 1000
# 정렬 옵션: sim (유사도순), date (날짜순)
sort = input('정렬 옵션 [sim (유사도순), date (날짜순)] : ')

news_df = pd.DataFrame(columns=('title', 'originallink', 'link', 'description', 'pubDate'))

for start_idx in range(start, end, display):    
    
    url = "https://openapi.naver.com/v1/search/news?query=" + query \
    + "&display=" + str(display) \
    + "&start=" + str(start_idx) \
    + "&sort=" + sort
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        
        for index in range(0, len(items)):
            tag_remover = re.compile('<.*?>')
            titles = re.sub(tag_remover, '', items[index]['title'])
            originallinks = items[index]['originallink']
            links = items[index]['link']
            descriptions = re.sub(tag_remover, '', items[index]['description'])
            pubDates = items[index]['pubDate']
            
            news_df.loc[idx] = [titles, originallinks, links, descriptions, pubDates]
            idx += 1
    else:
        print("Error Code:" + rescode)

client_id : 5vJNRZBYSgCMaC4WGGHD
client_secret : JmGndGBMUI
검색 키워드 : 전기차
정렬 옵션 [sim (유사도순), date (날짜순)] : sim


In [2]:
news_df.head()

,title,originallink,link,description,pubDate
0,"SK㈜, 이번엔 전기차 시장 공략…충전기 제조사 시그넷EV 인수(종합)",http://yna.kr/AKR20210415167800003?did=1195m,https://news.naver.com/main/read.nhn?mode=LSD&...,신주 포함 2천930억원으로 지분 55.5% 확보 스웨덴 전기차 폴스타에도 6천만달...,"Thu, 15 Apr 2021 18:04:00 +0900"
1,"SK, 전기차·급속 충전 시장 진출",http://www.etnews.com/20210415000266,https://news.naver.com/main/read.nhn?mode=LSD&...,SK㈜가 친환경 전기차 인프라 및 전기차 업체 투자로 세계 전기차 시장 공략에 나섰...,"Thu, 15 Apr 2021 18:23:00 +0900"
2,"SK, 전기차 시장 공략…시그넷 EV 인수·스웨덴 폴스타 투자",https://biz.chosun.com/site/data/html_dir/2021...,https://news.naver.com/main/read.nhn?mode=LSD&...,SK(034730)그룹의 투자전문회사 SK㈜가 국내 초급속 충전기 제조사인 시그넷 ...,"Thu, 15 Apr 2021 18:14:00 +0900"
3,제네시스·전기차·수소차 다 동원…中 시장 재건 나선 현대차그룹,http://news.mt.co.kr/mtview.php?no=20210415142...,https://news.naver.com/main/read.nhn?mode=LSD&...,현대차그룹이 전용 전기차를 비롯한 친환경차 라인업 확대 등 중국시장 재도약을 위한 ...,"Thu, 15 Apr 2021 15:00:00 +0900"
4,"SK, 전기차 충전기 제조사 '시그넷 EV' 인수… 스웨덴 전기차 '폴스타' 투자도",http://moneys.mt.co.kr/news/mwView.php?no=2021...,https://news.naver.com/main/read.nhn?mode=LSD&...,투자전문회사 SK가 친환경 전기차 인프라 및 기술 선점을 통해 글로벌 전기차 시장 ...,"Thu, 15 Apr 2021 18:14:00 +0900"


In [3]:
news_df.to_csv('data/news_electrcicar.csv', index=False)